In [1]:
# Load msd data. Takes about 6 minutes on CSEL-CUDA-03.cselabs.umn.edu

from datasets.msd import MSD

msd_data_config = {
    "name": "msd",
    "rewrite": False,
}

msd_dataset = MSD.from_config(msd_data_config)

2024-04-29 12:02:33,059 : [1/3] DATASET : Loading processed dataset datasets/data/msd/dataset.parquet.


In [2]:
msd_split_config = {
    "n_val_users": 50000,
    "n_test_users": 50000,
    "seed": 42,
    "target_proportion": 0.2,
    "targets_newest": False,
}

(msd_train, msd_val, msd_test), msd_split_time = msd_dataset.create_splits(msd_split_config)

2024-04-29 12:02:57,082 : [1/3] DATASET : Dataframe lengths | train_df: 27728200, val_df: 2953824, test_df: 2951425
2024-04-29 12:04:39,638 : [1/3] DATASET : Splits information:
2024-04-29 12:04:39,642 : [1/3] DATASET : Train split info | n_users = 471355, n_items = 41140, n_ratings = 27728200, sparsity = 99.86%
2024-04-29 12:04:39,644 : [1/3] DATASET : Validation split info | n_users = 50000, n_items = 41140, n_ratings = 2382298, sparsity = 99.88%
2024-04-29 12:04:39,645 : [1/3] DATASET : Test split info | n_users = 50000, n_items = 41140, n_ratings = 2380405, sparsity = 99.88%
2024-04-29 12:04:39,646 : [1/3] DATASET : Execution of create_splits took at 125.956 seconds.


In [3]:
# Create dictionary of item occurences for novelty metric
training_csr_matrix = msd_train.get_csr_matrix()
item_occurrences = training_csr_matrix.sum(axis=0)
item_ids = item_occurrences.nonzero()[1]
# Get the occurrences as a numpy array
item_occurrences = item_occurrences.A1
    
# Create a dictionary of item IDs and their occurrences
item_occurrences_dict = dict(zip(item_ids, item_occurrences))

In [4]:
from models.randomrec import RANDOM_REC

# No need to train, just predict
ran_pred = RANDOM_REC(msd_test.data, 20)

In [5]:
import recmetrics

msd_novelty, msd_novelty_topn = recmetrics.novelty(ran_pred, item_occurrences_dict, len(msd_test.data['user_id'].unique().tolist()), 20)

msd_novelty

6.9698148781674085